# ============================================================================
# DATA CLEANING & PREPARATION
# ============================================================================

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)


# 1. LOAD RAW DATA

In [17]:
print("\n1. LOADING RAW DATA")
print("-" * 80)

df = pd.read_csv('../data/raw/fraudTrain.csv')
print(f"Original dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

# Create a copy for cleaning
df_clean = df.copy()



1. LOADING RAW DATA
--------------------------------------------------------------------------------


Original dataset shape: 1,296,675 rows × 23 columns


# 2. REMOVE UNNECESSARY COLUMNS

In [18]:
print("\n2. REMOVING UNNECESSARY COLUMNS")
print("-" * 80)

# Remove the unnamed index column
if 'Unnamed: 0' in df_clean.columns:
    df_clean = df_clean.drop('Unnamed: 0', axis=1)
    print("✓ Removed 'Unnamed: 0' column")

print(f"✓ New shape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")


2. REMOVING UNNECESSARY COLUMNS
--------------------------------------------------------------------------------
✓ Removed 'Unnamed: 0' column
✓ New shape: 1,296,675 rows × 22 columns


# 3. CHECK FOR DUPLICATES

In [19]:
print("\n3. CHECKING FOR DUPLICATES")
print("-" * 80)

duplicates = df_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicates:,}")

if duplicates > 0:
    df_clean = df_clean.drop_duplicates()
    print(f"✓ Removed {duplicates:,} duplicate rows")
else:
    print("✓ No duplicates found")

print(f"✓ Shape after duplicate removal: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")

# Check for duplicate transaction numbers
dup_trans = df_clean['trans_num'].duplicated().sum()
print(f"\nDuplicate transaction numbers: {dup_trans:,}")


3. CHECKING FOR DUPLICATES
--------------------------------------------------------------------------------


Number of duplicate rows: 0
✓ No duplicates found
✓ Shape after duplicate removal: 1,296,675 rows × 22 columns

Duplicate transaction numbers: 0


# 4. DATA TYPE CONVERSIONS

In [20]:
print("\n4. DATA TYPE CONVERSIONS")
print("-" * 80)

print("\n📅 Converting datetime columns...")

# Convert trans_date_trans_time to datetime
df_clean['trans_date_trans_time'] = pd.to_datetime(df_clean['trans_date_trans_time'])
print("✓ Converted 'trans_date_trans_time' to datetime")

# Convert dob to datetime
df_clean['dob'] = pd.to_datetime(df_clean['dob'])
print("✓ Converted 'dob' to datetime")

# Ensure cc_num is string (not numeric)
df_clean['cc_num'] = df_clean['cc_num'].astype(str)
print("✓ Converted 'cc_num' to string")

# Ensure trans_num is string
df_clean['trans_num'] = df_clean['trans_num'].astype(str)
print("✓ Converted 'trans_num' to string")

# Ensure categorical columns are proper type
categorical_cols = ['merchant', 'category', 'first', 'last', 'gender', 'street', 
                    'city', 'state', 'job']
for col in categorical_cols:
    df_clean[col] = df_clean[col].astype('category')
print(f"✓ Converted {len(categorical_cols)} columns to category type")

print("\n📊 Current data types:")
print(df_clean.dtypes)


4. DATA TYPE CONVERSIONS
--------------------------------------------------------------------------------

📅 Converting datetime columns...


✓ Converted 'trans_date_trans_time' to datetime
✓ Converted 'dob' to datetime
✓ Converted 'cc_num' to string
✓ Converted 'trans_num' to string
✓ Converted 9 columns to category type

📊 Current data types:
trans_date_trans_time    datetime64[us]
cc_num                              str
merchant                       category
category                       category
amt                             float64
first                          category
last                           category
gender                         category
street                         category
city                           category
state                          category
zip                               int64
lat                             float64
long                            float64
city_pop                          int64
job                            category
dob                      datetime64[us]
trans_num                           str
unix_time                         int64
merch_lat                       flo

# 5. FEATURE ENGINEERING - TIME-BASED FEATURES

In [21]:
print("\n5. FEATURE ENGINEERING - TIME-BASED FEATURES")
print("-" * 80)

# Extract time components
df_clean['transaction_hour'] = df_clean['trans_date_trans_time'].dt.hour
df_clean['transaction_day'] = df_clean['trans_date_trans_time'].dt.day_name()
df_clean['transaction_day_of_week'] = df_clean['trans_date_trans_time'].dt.dayofweek
df_clean['transaction_month'] = df_clean['trans_date_trans_time'].dt.month
df_clean['transaction_year'] = df_clean['trans_date_trans_time'].dt.year
df_clean['transaction_date'] = df_clean['trans_date_trans_time'].dt.date

print("✓ Created: transaction_hour")
print("✓ Created: transaction_day (day name)")
print("✓ Created: transaction_day_of_week (0=Monday, 6=Sunday)")
print("✓ Created: transaction_month")
print("✓ Created: transaction_year")
print("✓ Created: transaction_date")

# Weekend flag
df_clean['is_weekend'] = (df_clean['transaction_day_of_week'] >= 5).astype(int)
print("✓ Created: is_weekend (1=Saturday/Sunday, 0=Weekday)")

# Geographic features: distance to merchant (km)
if 'distance_from_merchant_km' not in df_clean.columns:
    required_cols = {'lat', 'long', 'merch_lat', 'merch_long'}
    if required_cols.issubset(df_clean.columns):
        # Vectorized Haversine distance
        lat1 = np.radians(df_clean['lat'].astype(float))
        lon1 = np.radians(df_clean['long'].astype(float))
        lat2 = np.radians(df_clean['merch_lat'].astype(float))
        lon2 = np.radians(df_clean['merch_long'].astype(float))

        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
        c = 2 * np.arcsin(np.sqrt(a))
        earth_radius_km = 6371.0
        df_clean['distance_from_merchant_km'] = earth_radius_km * c
        print("✓ Created: distance_from_merchant_km (Haversine distance)")
    else:
        missing = sorted(required_cols - set(df_clean.columns))
        df_clean['distance_from_merchant_km'] = np.nan
        print(f"⚠️ Could not compute distance_from_merchant_km (missing columns: {missing}). Set to NaN.")

# Geographic flags (only if distance is available)
if df_clean['distance_from_merchant_km'].notna().any():
    df_clean['is_local_transaction'] = (df_clean['distance_from_merchant_km'] <= 50).astype(int)
    df_clean['is_far_transaction'] = (df_clean['distance_from_merchant_km'] > 200).astype(int)
    print("✓ Created: is_local_transaction (1=≤50km, 0=>50km)")
    print("✓ Created: is_far_transaction (1=>200km, 0=≤200km)")
else:
    df_clean['is_local_transaction'] = np.nan
    df_clean['is_far_transaction'] = np.nan
    print("⚠️ Skipped: is_local_transaction / is_far_transaction (distance_from_merchant_km is NaN)")

# Night transaction flag (10 PM - 6 AM)
df_clean['is_night'] = ((df_clean['transaction_hour'] >= 22) | 
                        (df_clean['transaction_hour'] < 6)).astype(int)
print("✓ Created: is_night (1=Night 10PM-6AM, 0=Day)")


5. FEATURE ENGINEERING - TIME-BASED FEATURES
--------------------------------------------------------------------------------


✓ Created: transaction_hour
✓ Created: transaction_day (day name)
✓ Created: transaction_day_of_week (0=Monday, 6=Sunday)
✓ Created: transaction_month
✓ Created: transaction_year
✓ Created: transaction_date
✓ Created: is_weekend (1=Saturday/Sunday, 0=Weekday)
✓ Created: distance_from_merchant_km (Haversine distance)
✓ Created: is_local_transaction (1=≤50km, 0=>50km)
✓ Created: is_far_transaction (1=>200km, 0=≤200km)
✓ Created: is_night (1=Night 10PM-6AM, 0=Day)


# 6. FEATURE ENGINEERING - AMOUNT-BASED FEATURES

In [22]:
print("\n6. FEATURE ENGINEERING - AMOUNT-BASED FEATURES")
print("-" * 80)

# Round amount flag (e.g., $50, $100, $500)
df_clean['amount_rounded'] = df_clean['amt'].round(0)
df_clean['is_round_amount'] = (df_clean['amt'] == df_clean['amount_rounded']).astype(int)
print("✓ Created: is_round_amount (1=Round number like $50, 0=Not round)")

# Amount categories
def categorize_amount(amt):
    if amt < 10:
        return 'Very Small'
    elif amt < 50:
        return 'Small'
    elif amt < 100:
        return 'Medium'
    elif amt < 500:
        return 'Large'
    else:
        return 'Very Large'

df_clean['amount_category'] = df_clean['amt'].apply(categorize_amount)
print("✓ Created: amount_category (Very Small/Small/Medium/Large/Very Large)")

# Log transformation of amount (useful for analysis)
df_clean['amt_log'] = np.log1p(df_clean['amt'])
print("✓ Created: amt_log (log-transformed amount for analysis)")


6. FEATURE ENGINEERING - AMOUNT-BASED FEATURES
--------------------------------------------------------------------------------
✓ Created: is_round_amount (1=Round number like $50, 0=Not round)
✓ Created: amount_category (Very Small/Small/Medium/Large/Very Large)
✓ Created: amt_log (log-transformed amount for analysis)


In [23]:
df_clean

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,transaction_hour,transaction_day,transaction_day_of_week,transaction_month,transaction_year,transaction_date,is_weekend,distance_from_merchant_km,is_local_transaction,is_far_transaction,is_night,amount_rounded,is_round_amount,amount_category,amt_log
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.08,-81.18,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.01,-82.05,0,0,Tuesday,1,1,2019,2019-01-01,0,78.60,0,0,1,5.00,0,Very Small,1.79
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.89,-118.21,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.16,-118.19,0,0,Tuesday,1,1,2019,2019-01-01,0,30.21,1,0,1,107.00,0,Large,4.68
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.18,-112.26,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.15,-112.15,0,0,Tuesday,1,1,2019,2019-01-01,0,108.21,0,0,1,220.00,0,Large,5.40
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.23,-112.11,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.03,-112.56,0,0,Tuesday,1,1,2019,2019-01-01,0,95.67,0,0,1,45.00,1,Small,3.83
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.42,-79.46,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.67,-78.63,0,0,Tuesday,1,1,2019,2019-01-01,0,77.56,0,0,1,42.00,0,Small,3.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,Hatch,UT,84735,37.72,-112.48,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.84,-111.69,0,12,Sunday,6,6,2020,2020-06-21,1,119.75,0,0,0,16.00,0,Small,2.81
1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,Tuscarora,MD,21790,39.27,-77.51,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.91,-78.25,0,12,Sunday,6,6,2020,2020-06-21,1,75.10,0,0,0,52.00,0,Medium,3.96
1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,High Rolls Mountain Park,NM,88325,32.94,-105.82,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.62,-105.13,0,12,Sunday,6,6,2020,2020-06-21,1,99.05,0,0,0,106.00,0,Large,4.67
1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,Manderson,SD,57756,43.35,-102.54,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.79,-103.24,0,12,Sunday,6,6,2020,2020-06-21,1,84.63,0,0,0,75.00,0,Medium,4.33


In [24]:
# Check for coordinate columns
coord_cols = [col for col in df_clean.columns if any(word in col.lower() for word in ['lat', 'long', 'merch'])]
print("Coordinate columns found:")
print(coord_cols)

Coordinate columns found:
['merchant', 'lat', 'long', 'merch_lat', 'merch_long', 'distance_from_merchant_km']
